# Naver 영화별 Review Crawling

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup  

import pandas as pd

from tqdm import tqdm_notebook
import re

## Naver 영화 - 2020/03/01 평점순 영화별 제목, Review, 장르 Crawling

https://movie.naver.com/movie/sdb/rank/rmovie.nhn

In [12]:
domain = 'https://movie.naver.com'
story = []
title = []
genre = []

for i in tqdm_notebook( range ( 1, 11 ) ):
    url = "https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&date=20200301&page=" + str( i )
    html = urlopen( url )
    soup = BeautifulSoup( html, "lxml" )
    titles = soup.find_all( 'div', class_ = 'tit5' )
    hype = []
    href = []
  
    try:
        for each in titles:  
            hype = each.find_all( 'a' )
            for link in hype:
                href.append( link[ 'href' ] )
                
        for j in tqdm_notebook( range( len( href ) ) ):
            domain = 'https://movie.naver.com'
            domain = domain + href[ j ]
            html = urlopen( domain )
            soup = BeautifulSoup( html,  "lxml" )
            story.append( soup.find( 'p', class_ = "con_tx" ).get_text() )
            
            title_tag = soup.find( 'h3', class_ = 'h_movie' )
            title.append( title_tag.find( 'a' ).get_text() )
            
            genre_tag = soup.find( 'p' )
            genre.append( genre_tag.find( 'a' ).get_text() )
    except:
        pass
                  

# 줄거리 정규화 처리
for i in tqdm_notebook( range( len( story ) ) ):
    story[i] = re.sub( '[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》“”’]', '', story[ i ] )
    story[i] = re.sub( '\r\xa0', '', story[ i ] )

d:\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


d:\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


d:\Anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:37: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


## Pandas DataFrame 생성

In [13]:
movie = pd.DataFrame( data = { 'title': title, 'story': story, 'genre': genre } )

In [20]:
movie.shape

(384, 3)

In [14]:
movie.head()

,title,story,genre
0,그린 북,1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아...,드라마
1,가버나움,나를 세상에 태어나게 한 부모님을 고소하고 싶어요출생기록조차 없이 살아온 어쩌면 1...,드라마
2,베일리 어게인,귀여운 소년 이든의 단짝 반려견 베일리는 행복한 생을 마감한다하지만 눈을 떠보니 다...,모험
3,주전장,일본의 인종차별 문제를 다룬 영상을 올린 후 우익들의 공격 대상이 된 일본계 미국인...,다큐멘터리
4,포드 V 페라리,1960년대 매출 감소에 빠진 포드는 판매 활로를 찾기 위해스포츠카 레이스를 장악한...,액션


In [15]:
movie.tail()

,title,story,genre
379,바그다드 카페 : 디렉터스컷,황량한 사막 한가운데 자리 잡은 초라한 바그다드 카페 커피머신은 고장난지 오래고 먼...,코미디
380,론 서바이버,2005년 6월 28일 아프가니스탄에서 복무중인 네이비씰 대원 마커스 마이클 대니 ...,액션
381,국제시장,1950년대 한국전쟁 이후로부터 현재에 이르기까지 격변의 시대를 관통하며 살아온 우...,드라마
382,말레나,2차 대전이 한창인 햇빛 찬란한 지중해의 작은 마을 매혹적인 말레나 걸어갈 때면 어...,드라마
383,콜드 마운틴,남군 병사인 인만주드 로은 전투 중에 중상을 입고 병원에 입원해있던 도중에 탈영을 ...,드라마


## csv로 저장하기 

In [23]:
movie.to_csv( 'movie_reviews.csv' )